# Time Series Analysis
---

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller  
from sklearn.model_selection import train_test_split

import pickle

import warnings
warnings.filterwarnings('ignore')

In [10]:
def var_model(first, last, mlbid):
    
    """
    Function to create a VAR model for each individual player
    Returns a pickle file of players model
    """
    
    base_path = '../data/clean_players/'
    
    #This string will be used to specifiy the player
    player_name = first + '-' + last + '-' + str(mlbid)
    
    #Full path to file
    file_path = base_path + player_name + '.csv'

   # try:
    df = pd.read_csv(file_path)

    train, test = train_test_split(df, test_size = 0.15, shuffle = False)

    model = VAR(train)

    model.fit()

    with open(f'../models/{first}-{last}-{mlbid}.pkl', 'wb') as pickle_out:
        pickle_out = pickle.dump(model, pickle_out)      
        
   # except:
       # print(f'{first} {last} something went wrong')

In [8]:
players = pd.read_csv('../data/mlb_players.csv').drop('Unnamed: 0', axis = 1)
players.head()

,MLBID,FIRSTNAME,LASTNAME,ACTIVE,Player,Team,Pos,Age,G,AB,...,BB,SO,SH,SF,HBP,AVG,OBP,SLG,OPS,salary
0,547989,Jose,Abreu,Y,Jose Abreu,CWS,1B,34,152,566,...,61,143,0,10,22,0.261,0.351,0.481,0.832,"$17,666,666"
1,642715,Willy,Adames,Y,Willy Adames,TB,SS,26,41,132,...,10,51,0,0,0,0.197,0.254,0.371,0.625,"$590,000"
2,501303,Ehire,Adrianza,Y,Ehire Adrianza,ATL,SS,32,109,182,...,21,42,1,3,2,0.247,0.327,0.401,0.728,"$1,500,000"
3,542583,Jesus,Aguilar,Y,Jesus Aguilar,MIA,1B,31,131,449,...,46,93,0,7,3,0.261,0.329,0.459,0.788,"$4,500,000"
4,605113,Nick,Ahmed,Y,Nick Ahmed,ARI,SS,31,129,434,...,34,104,2,1,2,0.221,0.280,0.339,0.619,"$8,125,000"


In [11]:
%%time
for index, row in players.iterrows():
    
    first = row['FIRSTNAME']
    last = row['LASTNAME']
    mlbid = row['MLBID']
    
    var_model(first, last, mlbid)
    
print('Models Finished')
    
# Copied from https://stackoverflow.com/questions/16476924/how-to-iterate-over-rows-in-a-dataframe-in-pandas

NameError: name 'pickle' is not defined

In [168]:
def ts_plot(lst):
    plt.figure(figsize=(16,10))
    
    for df in lst:
        plt.plot(df, label=df.columns[0])
        
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.legend();

In [171]:
columns = judge.columns
for column in columns:
    print(column)
    print(interpret_dftest(adfuller(judge[column])))

PA
p-value    0.272054
dtype: float64
AB
p-value    0.009276
dtype: float64
R
p-value    1.487501e-25
dtype: float64
H
p-value    0.0
dtype: float64
2B
p-value    0.0
dtype: float64
3B
p-value    0.0
dtype: float64
HR
p-value    9.008480e-29
dtype: float64
RBI
p-value    0.0
dtype: float64
BB
p-value    1.166265e-25
dtype: float64
SO
p-value    3.259949e-26
dtype: float64
AVG
p-value    0.000053
dtype: float64
OBP
p-value    0.000193
dtype: float64
SLG
p-value    0.000003
dtype: float64
OPS
p-value    0.000021
dtype: float64


In [14]:
df = pd.read_csv('../data/clean_players/Jesus-Aguilar-542583.csv')

In [15]:
df.shape

(462, 14)

In [17]:
var_model('Jesus', 'Aguilar', 542583)

LinAlgError: 6-th leading minor of the array is not positive definite